In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

_ = load_dotenv(find_dotenv())
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")


e:\LangChain project\learn-LangChain\learnlangchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Completion model

In [6]:
## Completion model
chatModel = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    google_api_key=GEMINI_API_KEY,
    temperature=0.2,
    max_output_tokens=4096
)

## Prompt templates


In [8]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
  "Tell me some {adjective} things of {topic} with some short sentences."
)

llmModelPrompt = prompt_template.format(
  adjective="curious",
  topic="John F Kennedy" 
)

response = chatModel.invoke(llmModelPrompt)
print(response.content)

*   **He was a prolific letter writer:** Kennedy penned thousands of letters throughout his life, revealing his thoughts and feelings.
*   **He had a love for sailing:** The sea was a source of relaxation and escape for him.
*   **He was a war hero:** Kennedy's bravery during World War II earned him the Navy and Marine Corps Medal.
*   **He had a reported fondness for Marilyn Monroe:** Their rumored relationship remains a subject of fascination.
*   **He was a Pulitzer Prize winner:** Kennedy won the Pulitzer Prize for his book "Profiles in Courage."



In [20]:
#Chat template
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a {profession} expert about {topic}."),
    ("user", "Hello, can you answer some questions about {topic}?"),
    ("assistant", "Sure, I can help you with that."),
    ("user", "{user_input}"),
  ]
)

message = chat_template.format_messages(
  profession="historian",
  topic="John F Kennedy",
  user_input="Tell me some curious things of John F Kennedy with some short sentences."
)
response = chatModel.invoke(message)
print(response.content)

Okay, here are some curious facts about John F. Kennedy:

*   **He was a prolific letter writer:** Kennedy wrote thousands of letters throughout his life, offering insights into his thoughts and feelings.
*   **He had a love for sailing:** Kennedy owned a sailboat named "Honey Fitz" and enjoyed spending time on the water.
*   **He was a skilled swimmer:** Kennedy was a strong swimmer and even saved a crew member during World War II.
*   **He had a penchant for reading:** Kennedy was known to read extensively, often carrying books with him.
*   **He was a fashion icon:** Kennedy's style, including his tailored suits and classic hairstyles, influenced men's fashion.
*   **He had a secret code name:** During his presidency, Kennedy's Secret Service code name was "Lancer."
*   **He was a Pulitzer Prize winner:** Kennedy won a Pulitzer Prize for his book "Profiles in Courage."
*   **He had a close relationship with his siblings:** Kennedy was very close to his siblings, especially his siste

## Fewshot prompting

In [17]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

examples = [
    { "input": "hi", "output": "Xin chào" },
    { "input": "bye", "output": "tạm biệt" },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("user", "{input}"),
        ("assistant", "{output}"), 
    ]
)

fewshot = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a translator from English to Vietnamese."),
        fewshot,
        ("user", "{input}"),
    ]
)

In [ ]:
message = final_prompt.format_messages(input="hello")

response = chatModel.invoke(message)
print(response.content)

AttributeError: 'str' object has no attribute 'invoke'

## Chain

In [21]:
chain = final_prompt | chatModel 
res2 = chain.invoke({"input": "how are you?"})
print(res2.content)

Bạn khỏe không?



In [ ]:
#Langchain expression Language

## Output Parser

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "return a json object with an 'answer' field that answer the question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | chatModel | json_parser

res = json_chain.invoke({"question": "What is the capital of Vietnam?"})
print(res)

{'answer': 'Hanoi'}


You can use Pydantic model to define a custom output format

In [23]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [24]:
# define a Pydantic model for the output
class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")

In [25]:
# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

# Add the parser format instructions in the prompt definition.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Create a chain with the prompt and the parser
chain = prompt | chatModel | parser

chain.invoke({"query": "Tell me a joke."})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

In [26]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "The setup of the joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "The punchline of the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```'